# Putting Redux (CT 8)

For this challenge task, you'll be asked to revisit the lag putting problem we did early on in the term.  But now we have the tools to create a much more realistic model: we can deal with the continuous changes in the slope of the green, we know how to deal with vectors, and we have a range of optimization tools at our disposal.

<br>

---

In [ ]:
#@title
# Import necessary libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

---

<br>

## The Challenge

Your goal is to create a simulation that can determine the velocity and direction of a putt that will stop at the hole.  The initial location of the ball and the location of the hole should be system parameters for an individual simulation (and should be adjustable for a later simulation).

<br> 

The only predetermined parameter that you will need is the coefficient of rolling friction ($\mu = 0.15$).  If you choose to incorporate drag, you can assume that the coefficient of drag is $C_d = 0.7$, the radius of the ball is $r = 2.16 cm$, and the density of air is $1.3 kg/m^3$.

## The Green

The details for the green are included here.  As you will see, the green data is formatted in a way that will allow you to access the slope of the green at any given coordinate point.  To help you visualize the green, here are two plots: the first of the elevation on the green, and the second of the slope of the green (You can look at this code to see where this is coming from, but don't necessarily need to):

In [ ]:
#@title
from scipy.interpolate import interpn
from numpy import gradient
import matplotlib.pyplot as plt

# Upload elevation data for the green and put data in an array
filename = 'https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Data/CT8_golf_green_data.xlsx'
data = pd.read_excel(filename, header=0, index_col=0)
green = np.array(data)

# Calculate the gradients at the green so that they can
# be accessed using coordinate values
grad = gradient(green)
grad[0] = np.flip(np.rot90(grad[0]),1)
grad[1] = np.flip(np.rot90(grad[1]),1)
grad = np.flip(grad)

# Plot a topographic map of the green
fig, (ax_topo, ax_slope) = plt.subplots(nrows=1, ncols=2, figsize=(14,10))
cs = ax_topo.contour(np.array(data.columns), np.array(data.index), green, levels=15)
ax_topo.clabel(cs,colors='black', fmt = '%1.3f');
ax_topo.set(title='Elevation Map of Green', xlabel='X-coordinate', ylabel='Y-coordinate');

# Plot the slope of the green (arrows pointing downhill)
x, y = np.meshgrid(np.array(data.index), np.array(data.columns))
ax_slope.quiver(y,x,-grad[0],-grad[1]);
ax_slope.set(title='Slope Map of Green', xlabel='X-coordinate');

Note that both the coordinate values and the elevation values are in meters.

<br>

The NumPy array `grad` (created in the cell above) holds the gradient values at a set of defined values.  For instance, we can access the slope of the green at the coordinate point (1,1) by doing the following:


In [ ]:
print(grad[0][1][1], grad[1][1][1])

Make sure you understand what these values mean by looking at the contour map above (and note that the slope map on the right shows the *downhill* slope).  

<br>

But `grad` has a limited usefulness:

<br>

* Its arguments refer to index numbers, rather than distances
* More importantly, since it can provide the slope only at a given set of data points, and you will need to access the slope at any plot on the green.  So we need to be able to find a slope by interpolating between known points. 

<br> 

The tool you can use to do that is the SciPy function `interpn`. `interpn` interpolates a x- and y- component of the slope for any point on the green.  This code returns the x-component of the slope at the point $(1.5 ~m, 1.5 ~m)$.

In [ ]:
points = (np.array(data.columns), np.array(data.index))
interpn(points,grad[0],[1.5,1.5])

`points` creates an array of the known points.  Once it is created, we can find the slope at any point on the green.  It returns the x-component of the slope if `grad[0]` is entered as the second argument; if `grad[1]` is the second argument, the return value is the y-component of the slope.

## How to Proceed

Here are some things to start with:

<br>

* To understand the physics of a ball rolling on uneven ground, you might look back at Challenge Task 1.

* Create a set of `params`, and a `make_system` function.  What values will you need to run a particular simulation?

* To understand the structure of the code and how to use vectors, you could start by looking at the baseball notebooks (3.9.1-3.9.3).  

* In particular, look at the code map of the baseball notebooks to understand the different parts of the code you'll need for the optimization process.